**Set environment**

In [1]:
suppressMessages(suppressWarnings(source("../config/config_sing.R")))
show_env()

You are in Singularity: singularity_proj_encode_fcc 
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei 
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/out 
CODE DIRECTORY (FD_CODE): /data/reddylab/Kuei/code 
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC 
PATH OF RESULTS (FD_RES): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc 
PATH OF LOG     (FD_LOG): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log 


**Check data**

In [2]:
FOLDER = "annotation_encode_e2g_prediction"

In [3]:
fdiry = file.path(
    FD_RES, 
    "results", 
    "region", 
    FOLDER)
for(fname in dir(fdiry)){print(fname)}

[1] "description.tsv"
[1] "K562.ENCSR328LMT.ENCFF122UMY.element_gene_links.total.bed.gz"
[1] "K562.ENCSR328LMT.ENCFF122UMY.element_gene_links.total.merge.bed.gz"
[1] "K562.ENCSR328LMT.ENCFF202FID.element_gene_links.active.bed.gz"
[1] "K562.ENCSR328LMT.ENCFF202FID.element_gene_links.active.merge.bed.gz"


In [4]:
fdiry = file.path(
    FD_RES, 
    "results", 
    "region", 
    "KS91_K562_ASTARRseq_peak_macs_input", 
    FOLDER)
for(fname in dir(fdiry)){print(fname)}

[1] "peak.annotation.K562.ENCSR328LMT.ENCFF122UMY.element_gene_links.total.bed.gz"
[1] "peak.annotation.K562.ENCSR328LMT.ENCFF122UMY.element_gene_links.total.merge.bed.gz"
[1] "peak.annotation.K562.ENCSR328LMT.ENCFF202FID.element_gene_links.active.bed.gz"
[1] "peak.annotation.K562.ENCSR328LMT.ENCFF202FID.element_gene_links.active.merge.bed.gz"


## Import data

In [5]:
fdiry  = file.path(FD_RES, "results", "region", FOLDER)
fname = "description.tsv"
fpath = file.path(fdiry, fname)

dat = read_tsv(fpath, show_col_types = FALSE)

dat_cnames = dat
print(dim(dat))
dat

[1] 11  2


Name,Description
<chr>,<chr>
Chrom,Chromosome of the perturbed element
Start,Start of the perturbed element (0-based)
End,End of the perturbed element
Name,"Identifier for the given E-G pair in the form of 'gene|chr:start-stop:strand', e.g. 'BET1L|chr11:220536-220704:*'"
Score,"Change in gene expression upon CRISPRi perturbation, e.g. precentage change in expression or log fold change"
Region_Class,"Type of region (intergenic, promoter, genic)"
Gene_ENSEMBL,target gene ensembl
Gene_Symbol,target gene symbol
Gene_TSS,target gene TSS


In [6]:
### init: set column names
cnames = dat_cnames$Name
cnames = c("Chrom_ATAC", "Start_ATAC", "End_ATAC", cnames, "Overlap")

### init: set file path
fdiry = file.path(
    FD_RES, 
    "results", 
    "region", 
    "KS91_K562_ASTARRseq_peak_macs_input", 
    FOLDER)
fname = paste("peak.annotation", ".active.bed.gz", sep="*")
fglob = file.path(fdiry, fname)
fpath = Sys.glob(fglob)

### init: annotation and label
annotation = "ENCODE-E2G"
label      = "ENCODE-E2G_Prediction"

### read data
dat = read_tsv(fpath, col_names = cnames, show_col_types = FALSE)
dat = dat %>% dplyr::mutate(
    Peak_ATAC  = paste0(Chrom_ATAC, ":", Start_ATAC, "-", End_ATAC),
    Annotation = annotation,
    Label      = label,
    Region     = paste0(Chrom, ":", Start, "-", End),
    Value      = Score,
    Note       = Region_Class
)

### assign and show
dat_peak_annot_import = dat
print(dim(dat))
head(dat)

[1] 104089     21


Chrom_ATAC,Start_ATAC,End_ATAC,Chrom,Start,End,Name,Score,Region_Class,Gene_ENSEMBL,⋯,Gene_TSS,Assay_Type,Assay_Label,Overlap,Peak_ATAC,Annotation,Label,Region,Value,Note
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,⋯,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>
chr1,115411,115986,chr1,115484,115984,chr1:115484-115984,0.41016,intergenic,ENSG00000187634,⋯,925740,ENCODE-E2G_Prediction,ENCODE-E2G_Prediction:SAMD11,500,chr1:115411-115986,ENCODE-E2G,ENCODE-E2G_Prediction,chr1:115484-115984,0.41016,intergenic
chr1,777949,779437,chr1,778438,779143,chr1:778438-779143,0.87131,promoter,ENSG00000177757,⋯,817370,ENCODE-E2G_Prediction,ENCODE-E2G_Prediction:FAM87B,705,chr1:777949-779437,ENCODE-E2G,ENCODE-E2G_Prediction,chr1:778438-779143,0.87131,promoter
chr1,777949,779437,chr1,778438,779143,chr1:778438-779143,0.70513,promoter,ENSG00000225880,⋯,827522,ENCODE-E2G_Prediction,ENCODE-E2G_Prediction:LINC00115,705,chr1:777949-779437,ENCODE-E2G,ENCODE-E2G_Prediction,chr1:778438-779143,0.70513,promoter
chr1,777949,779437,chr1,778438,779143,chr1:778438-779143,0.75133,promoter,ENSG00000228794,⋯,827590,ENCODE-E2G_Prediction,ENCODE-E2G_Prediction:LINC01128,705,chr1:777949-779437,ENCODE-E2G,ENCODE-E2G_Prediction,chr1:778438-779143,0.75133,promoter
chr1,777949,779437,chr1,778438,779143,chr1:778438-779143,0.99514,promoter,NA,⋯,778688,ENCODE-E2G_Prediction,ENCODE-E2G_Prediction:LOC100288069,705,chr1:777949-779437,ENCODE-E2G,ENCODE-E2G_Prediction,chr1:778438-779143,0.99514,promoter
chr1,777949,779437,chr1,778438,779143,chr1:778438-779143,0.61667,promoter,ENSG00000187634,⋯,925740,ENCODE-E2G_Prediction,ENCODE-E2G_Prediction:SAMD11,705,chr1:777949-779437,ENCODE-E2G,ENCODE-E2G_Prediction,chr1:778438-779143,0.61667,promoter


## Arrange and summarize

In [7]:
### init
dat = dat_peak_annot_import

### summarize
dat = dat %>%
    dplyr::group_by(Chrom_ATAC, Start_ATAC, End_ATAC, Peak_ATAC, Annotation, Label) %>%
    dplyr::summarise(
        Count  = n(), 
        Region = paste(Region, collapse="|"),
        Score  = mean(Value),
        Note   = paste(sort(unique(Note)), collapse="|"),
        .groups = "drop")

## Arrange
dat = dat %>% 
    dplyr::arrange(Chrom_ATAC, Start_ATAC, End_ATAC) %>%
    dplyr::select(-Chrom_ATAC, -Start_ATAC, -End_ATAC) %>%
    dplyr::rename("Peak" = Peak_ATAC)

### assign and show
dat_peak_annot_summary = dat
print(dim(dat))
head(dat)

[1] 35672     7


Peak,Annotation,Label,Count,Region,Score,Note
<chr>,<chr>,<chr>,<int>,<chr>,<dbl>,<chr>
chr1:115411-115986,ENCODE-E2G,ENCODE-E2G_Prediction,1,chr1:115484-115984,0.410160,intergenic
chr1:777949-779437,ENCODE-E2G,ENCODE-E2G_Prediction,5,chr1:778438-779143|chr1:778438-779143|chr1:778438-779143|chr1:778438-779143|chr1:778438-779143,0.787916,promoter
chr1:816774-817547,ENCODE-E2G,ENCODE-E2G_Prediction,3,chr1:817120-817620|chr1:817120-817620|chr1:817120-817620,0.602110,promoter
chr1:826754-828040,ENCODE-E2G,ENCODE-E2G_Prediction,5,chr1:827244-827847|chr1:827244-827847|chr1:827244-827847|chr1:827244-827847|chr1:827244-827847,0.713476,promoter
chr1:842480-843449,ENCODE-E2G,ENCODE-E2G_Prediction,1,chr1:842505-843350,0.370660,genic
chr1:903633-905556,ENCODE-E2G,ENCODE-E2G_Prediction,1,chr1:904203-905033,0.574120,intergenic


## Explore and check results

In [8]:
dat = dat_peak_annot_summary
lst = split(dat, dat$Label)
lst = lapply(lst, function(dat){
    table(dat$Note)
})
lst

$`ENCODE-E2G_Prediction`

                    genic          genic|intergenic genic|intergenic|promoter 
                    12610                        26                        16 
           genic|promoter                intergenic       intergenic|promoter 
                      891                     11011                       297 
                 promoter 
                    10821 


In [9]:
dat = dat_peak_annot_summary
lst = split(dat, dat$Label)
lst = lapply(lst, function(dat){
    table(dat$Count)
})
lst

$`ENCODE-E2G_Prediction`

    1     2     3     4     5     6     7     8     9    10    11    12    13 
16008  7432  3871  2421  1549  1076   732   538   404   324   243   179   139 
   14    15    16    17    18    19    20    21    22    23    24    25    26 
  129   111    89    69    56    49    34    36    25    21    14    16    12 
   27    28    29    30    31    32    33    34    35    36    37    38    39 
    9    14    12     8     2     8     8     3     5     2     1     4     1 
   40    42    43    46    47    48    58    85    86 
    3     4     3     1     3     1     1     1     1 


## Save results

In [10]:
fdiry = file.path(
    FD_RES, 
    "results", 
    "region", 
    "KS91_K562_ASTARRseq_peak_macs_input", 
    "summary")
fname = "peak.summary.encode_e2g_prediction.tsv"
fpath = file.path(fdiry, fname)

dat = dat_peak_annot_summary
write_tsv(dat, fpath)